# Extracción de datos Datasets Google Maps y YELP


### Importaciones


In [1]:
import os
import pandas as pd
import numpy as np
import json
import datetime
import geojson
from shapely.geometry import shape, Point
import shapely
from shapely.geometry import shape, Point
from shapely.errors import TopologicalError

### Definiciones y carpetas


Creación de diccionario de Estados


In [2]:
state_abreviations = ["AL","AK","AZ","AR","CA","CO","CT","DE","FL","GA","HI","ID","IL","IN","IA","KS","KY","LA","ME","MD","MA","MI","MN","MS","MO","MT","NE","NV","NH","NJ","NM","NY","NC","ND","OH","OK","OR","PA","RI","SC","SD","TN","TX","UT","VT","VA","WA","WV","WI","WY",]
state_dictionary = {"AL": "Alabama","AK": "Alaska","AZ": "Arizona","AR": "Arkansas","CA": "California","CO": "Colorado","CT": "Connecticut","DE": "Delaware","FL": "Florida","GA": "Georgia","HI": "Hawaii","ID": "Idaho","IL": "Illinois","IN": "Indiana","IA": "Iowa","KS": "Kansas","KY": "Kentucky","LA": "Louisiana","ME": "Maine","MD": "Maryland","MA": "Massachusetts","MI": "Michigan","MN": "Minnesota","MS": "Mississippi","MO": "Missouri","MT": "Montana","NE": "Nebraska","NV": "Nevada","NH": "New Hampshire","NJ": "New Jersey","NM": "New Mexico","NY": "New York","NC": "North Carolina","ND": "North Dakota","OH": "Ohio","OK": "Oklahoma","OR": "Oregon","PA": "Pennsylvania","RI": "Rhode Island","SC": "South Carolina","SD": "South Dakota","TN": "Tennessee","TX": "Texas","UT": "Utah","VT": "Vermont","VA": "Virginia","WA": "Washington","WV": "West Virginia","WI": "Wisconsin","WY": "Wyoming",}

Creación de directorios si no existen


In [3]:
os.mkdir("Generated") if not os.path.exists("Generated") else None
os.chdir("Generated")
for subdir in ["Google", "Yelp", "Unificado"]:os.mkdir(subdir) if not os.path.exists(subdir) else None
os.chdir("../")

# Extracción con los datasets de Google Maps


## Metada de Sitios


Se recorren los archivos línea por línea, durante el proceso se almacenan sólo las filas que incluyan <code>Restaurant</code> en la columna de categoría, así se optimiza el tamaño del dataset final.


In [4]:
lineas_json = []

for i in range(1, 12):
    path = f"Datasets/Google Maps/metadata-sitios/{i}.json"
    with open(path, "r") as file:
        for l in file:
            try:
                linea_j = json.loads(l)
                if "restaurant" in " ".join(linea_j["category"]).lower():
                    lineas_json.append(linea_j)
            except: pass

df = pd.DataFrame(lineas_json)
df.to_parquet(r"Generated\Google\metada_sitios.parquet")

Tamaño Directorio <code>metadata-sitios</code>: 2.76 Gb

Tamaño Archivo <code>metada_sitios.parquet</code>: 60.43 Mb

Dimensiones: 212.014 filas x 15 Columnas


### Obtención de información de Estados


En base al campo <code>Address</code> obtenemos el estado donde se encuentra el negocio. Nos servirá para luego seleccionar los estados con más restaurantes.


In [5]:
def get_state_ab(st):
    try:
        state = st.split(", ")[-1].split(" ")[0]
        if state in state_abreviations: return state
        else: return np.nan
    except: return np.nan

df["state_ab"] = df["address"].apply(get_state_ab)
top_5 = df["state_ab"].value_counts().head(5).index.to_list()
df["us_state"] = df["state_ab"].map(state_dictionary)

top_5_url = [
    f"Datasets/Google Maps/reviews-estados/review-{state_dictionary[i].replace(' ', '_')}/"
    for i in top_5]

cantidad_archivos = {}
for i in top_5_url:
    for j in os.walk(i):
        cantidad_archivos[i] = len(j[2])

## Reviews Estados


Ya con los estados elegidos estamos en condiciones de ingestar los datos de las carpetas correspondientes dentro del directorio <code>reviews-estados</code>.
Es información masiva lo que genera un archivo de grandes dimensiones, sin embargo previamente filtramos por el parámetro de año <code>2017-2019</code> valiéndonos del campo <code>time</code>, que tiene es un <code>timestamp</code>, pero con 3 digitos más que el usado por <code>datetime</code> de Python. Le agregamos el campo <code>Estado</code> que es más descriptivo.


In [6]:
lineas_json_revs_google = []

for i in top_5_url:
    count = 0
    for c in range(1, cantidad_archivos[i] + 1):
        file_name = f"{i}{c}.json"
        #file_name = f"Datasets/Google Maps/reviews-estados/{i}{c}.json"
        if os.path.exists(file_name):
            with open(file_name, "r", encoding="utf-8") as f:
                for s in f:
                    linea = json.loads(s)
                    linea["anio"] = datetime.datetime.fromtimestamp(linea["time"] / 1000).year
                    linea["estado"] = i.split("-")[-1][:-1]
                    if linea["anio"] in [2017, 2018, 2019]:
                        lineas_json_revs_google.append(linea)
        else:
            print(f"El archivo {file_name} no existe.")
        count += 1

df_revs_google = pd.DataFrame(lineas_json_revs_google)
merge_site_reviews = pd.merge(df_revs_google, df, left_on="gmap_id", right_on="gmap_id")
merge_site_reviews.to_parquet(r"Generated\Google\merge_site_reviews.parquet")
df_revs_google.to_parquet(r"Generated\Google\reviews-estados.parquet")

El archivo Datasets/Google Maps/reviews-estados/review-California/19.json no existe.
El archivo Datasets/Google Maps/reviews-estados/review-Texas/17.json no existe.
El archivo Datasets/Google Maps/reviews-estados/review-New_York/19.json no existe.
El archivo Datasets/Google Maps/reviews-estados/review-Florida/20.json no existe.
El archivo Datasets/Google Maps/reviews-estados/review-Pennsylvania/17.json no existe.


Tamaño archivo: 760 Mb

Tamaño dataset: 24.3 Gb

Tamaño 8.339.179 filas x 10 Columnas.


# Extracción de los Dataset de YELP


## Business


Contiene los datos de las entidades negocios de Yelp, a un primer vistazo tiene las columnas duplicadas, por lo que hay que hacer un recorte, ya que la segunda mitad tiene datos vacíos en su inmensa mayoría.


In [7]:
url_business = r"Datasets\Yelp\business.pkl"
df_business = pd.read_pickle(url_business)
df_business = df_business.iloc[:, :-14]
df_business = df_business[df_business.state.isin(top_5)]

def is_restaurant(st):
    try:
        test = "".join(st).lower()
        return "restaurant" in test
    except: return False

df_business = df_business[df_business["categories"].apply(is_restaurant)]
df_business.to_parquet(r"Generated\Yelp\bussines.parquet")

### Checkin


Se decidió que los datos de registros en el negocio por parte de los usuarios no era información pertinente para el proyecto, por lo que el archivo "checkin.json" no será cargado


### Tips


Se decidió que los datos de consejos de los usuarios no era información pertinente para el proyecto, por lo que el archivo "tip.json" no será cargado


### Review


In [8]:
df_reviews_url = r"Datasets\Yelp\review.json"
lineas_json_review = []

with open(df_reviews_url, "r", encoding="utf-8") as f:
    count = 0
    for i in f:
        linea = json.loads(i)
        anio = linea["date"][:4]
        if anio in ["2017", "2018", "2019"] and linea["useful"] == 1:
            lineas_json_review.append(linea)

df_reviews = pd.DataFrame(lineas_json_review)
df_reviews["funny"] = df_reviews["funny"].astype("int8")
df_reviews["stars"] = df_reviews["stars"].astype("int8")
df_reviews["cool"] = df_reviews["cool"].astype("int8")
df_reviews.drop("useful", axis=1, inplace=True, errors="ignore")

df_reviews.to_parquet(r"Generated\Yelp\review.parquet")

### Users Yelp


Se decidió que los datos directamente relacionados a los usuarios no era información pertinente para el proyecto, por lo que el archivo "users.parquet" no será cargado


# Recarga de dataframes


Se realiza una carga de los dataframes exportados previamente, para acelerar el proceso de testeo, ya que los pasos anteriores requieren de un alto consumo de tiempo.


In [9]:
df_maps_restaurantes = pd.read_parquet(r"Generated\Google\metada_sitios.parquet")
df_maps_reviews = pd.read_parquet(r"Generated\Google\merge_site_reviews.parquet")
df_yelp_restaurantes = pd.read_parquet(r"Generated\Yelp\bussines.parquet")
df_yelp_reviews = pd.read_parquet(r"Generated\Yelp\review.parquet")

# Limpieza de columnas


Se mantendrán sólo las columnas relacionadas al Modelo Entidad Relación, al mismo tiempo, serán renombradas para tener un formato unificado, que permita la unión en un sólo dataframe según las tablas del MER.

- Se crea df_maps_reviews.review_id tomando los primeros 10 caracteres de gmap_id y user_id


In [ ]:
df_maps_restaurantes = df_maps_restaurantes[["name","gmap_id","category","num_of_reviews","latitude","longitude","MISC","avg_rating",]]
df_maps_restaurantes = df_maps_restaurantes.rename(
    columns={
        "name": "nombre",
        "gmap_id": "id_restaurante",
        "category": "categorias",
        "num_of_reviews": "cantidad_resenas",
        "latitude": "latitud",
        "longitude": "longitud",
        "MISC": "atributos",
        "avg_rating": "calificacion",})

df_yelp_restaurantes = df_yelp_restaurantes[["name","business_id","categories","review_count","latitude","longitude","attributes","stars",]]
df_yelp_restaurantes = df_yelp_restaurantes.rename(
    columns={
        "name": "nombre",
        "business_id": "id_restaurante",
        "categories": "categorias",
        "review_count": "cantidad_resenas",
        "latitude": "latitud",
        "longitude": "longitud",
        "attributes": "atributos",
        "stars": "calificacion",})

df_maps_reviews["review_id"] = (
    df_maps_reviews["gmap_id"].str[:10] + df_maps_reviews["user_id"].str[:10])
df_maps_reviews["sentiment_score"] = 0
df_maps_reviews = df_maps_reviews[["user_id","gmap_id","review_id","rating","anio","sentiment_score",]]
df_maps_reviews = df_maps_reviews.rename(
    columns={
        "user_id": "id_usuario",
        "gmap_id": "id_restaurante",
        "review_id": "id_resena",
        "rating": "calificacion",
        "anio": "anio",
        "sentiment_score": "puntaje_de_sentimiento",})

df_yelp_reviews["anio"] = df_yelp_reviews["date"].str[:4]
df_yelp_reviews["sentiment_score"] = 0
df_yelp_reviews = df_yelp_reviews[["user_id","business_id","review_id","stars","anio","sentiment_score",]]
df_yelp_reviews = df_yelp_reviews.rename(
    columns={
        "user_id": "id_usuario",
        "business_id": "id_restaurante",
        "review_id": "id_resena",
        "stars": "calificacion",
        "anio": "anio",
        "sentiment_score": "puntaje_de_sentimiento",})

# Unión de Dataframes y limpieza de duplicados


Se unirán los Dataframes de restaurantes, y los dataframes de reviews, además de esto, se hará una revisión de los posibles registros duplicados y se eliminarán, esto apoyandose en los datos de longitud y latitud


In [ ]:
df_restaurantes = pd.concat([df_yelp_restaurantes, df_maps_restaurantes])
df_restaurantes = df_restaurantes.drop_duplicates(subset=["id_restaurante"])
# Corrección de columna categorias
def convert_to_string(value):
    if isinstance(value, np.ndarray): 
        return ", ".join(map(str, value))
    return str(value)
df_restaurantes["categorias"] = df_restaurantes["categorias"].apply(convert_to_string)

df_reviews = pd.concat([df_yelp_reviews, df_maps_reviews])
df_reviews = df_reviews.drop_duplicates(subset=["review_id"])
# corrección de columna anio
df_reviews["anio"] = df_reviews["anio"].astype(int)

# Creación de Dataframe Estados


por necesidades del proyecto, se creará un dataframe con la información de los estados, para ser consultado, cargado y consumido.

El dataset elegido para conseguir los datos de densidad de población contiene la población del último censo, del año 2020, debe actualizarse con el nuevo censo.


In [ ]:
df_estados = pd.DataFrame(
    {
        "State Abbreviation": state_abreviations,
        "State": [state_dictionary[abrv] for abrv in state_abreviations],
        "Population": [0] * len(state_abreviations),})

df_poblacion = pd.read_csv("Datasets/us_pop_by_state.csv")
for index, row in df_poblacion.iterrows():
    state_code = row["state_code"]
    population = row["2020_census"]
    if state_code in df_estados["State Abbreviation"].values:
        df_estados.loc[df_estados["State Abbreviation"] == state_code, "Population"] = (
            population)

# Corrección de Estados según Geolocalización

In [ ]:
state_dictionary_inv = {v: k for k, v in state_dictionary.items()}

def get_state_from_geojson(lat, lon, data_geo):
    for feature in data_geo['features']:
        polygon = shape(feature['geometry'])
        # Verificar las diferentes estructuras de propiedades
        if 'NAME' in feature['properties']:
            state_name = feature['properties']['NAME']
        elif 'name' in feature['properties']:
            state_name = feature['properties']['name']
        else:
            continue
        try:
            if polygon.contains(Point(lon, lat)):
                return state_dictionary_inv.get(state_name, None)
        except TopologicalError as e:
            print(f"TopologicalError: {e} at ({lat}, {lon})")
            return None
    return None


def get_state_1(lat, lon): return get_state_from_geojson(lat, lon, data_geo_1)

def get_state_2(lat, lon): return get_state_from_geojson(lat, lon, data_geo_2)

def safe_get_state(row): 
    try: return get_state_1(row['latitud'], row['longitud']) or get_state_2(row['latitud'], row['longitud'])
    except TopologicalError as e:
        print(f"Skipping row due to TopologicalError: {e}")
        return None

In [ ]:
df_restaurantes['state_ab'] = df_restaurantes.apply(safe_get_state, axis=1)
print(df_restaurantes.head())

GEOSException: TopologyException: side location conflict at -135.61113979463926 57.425339066329983. This can occur if the input geometry is invalid.

# Extracción de Dataframes finales


In [ ]:
try:
    df_restaurantes_existente = pd.read_parquet(r"Generated\Unificado\restaurantes.parquet")
    df_reviews_existente = pd.read_parquet(r"Generated\Unificado\reviews.parquet")
except FileNotFoundError:
    df_restaurantes_existente = pd.DataFrame()
    df_reviews_existente = pd.DataFrame()

df_restaurantes_concat = pd.concat([df_restaurantes_existente, df_restaurantes])
df_reviews_concat = pd.concat([df_reviews_existente, df_reviews])

df_restaurantes_concat.to_parquet(r"Generated\Unificado\restaurantes.parquet")
df_reviews_concat.to_parquet(r"Generated\Unificado\reviews.parquet")
df_estados.to_parquet(r"Generated\Unificado\estados.parquet")